## Hydrology 

- ETP comparison

In [1]:
import datetime
import os

import plotly.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go
init_notebook_mode(connected=True)

import numpy as np

from obs_data import *

print(os.getcwd())

2016
D:\Documents\these_pablo\Models\BEACH2016\Analysis


In [2]:
PC = True
analysis = False
short_sim = True
version = "model_v12"
folder = "1"
timestamp = 'julian'

In [3]:
if analysis:
    dot = ''
else:
    dot = '..\\'
    
if PC:
    back = "\\"  
    sim_path = dot + version + back  
else:
    back = "/"
    sim_path = version + back
    
print(sim_path)

..\model_v12\


## Observed

In [4]:
# ETP (mm -> m3)
etp_obs = getETP()
etp_m3 = etp_obs.ETPmm*47*(10**4)/10**3

In [5]:
if short_sim:
    s_obs = 166 - 1
    e_obs = 300
else:
    # Julian days
    s_obs = 0
    e_obs = 300

# Calendar days
if timestamp == 'julian':
    # Julian days
    x = etp_obs.Jdays.tolist()[s_obs:e_obs]
else:
    # Calendar days
    dates = getTime(sep=";")
    x = dates.Date.tolist()[s_obs:e_obs]
    # dates = getTime(sep=";")
    # x = dates.Date.tolist()[s:e]

### Import

In [6]:
# ETP sim
sims = ['accEtp_m3', 'accTransp_m3', 'accEvap_m3']
names = ['ETP Sim.', 'Transp Sim.', 'Evap Sim.']
palette = ['blue', 'purple', '#F44336']

data = []  # <- store all graphing data

etpObs = etp_m3.tolist()[s_obs:e_obs]
traceObs = go.Scatter(
    x = x,
    y = etpObs,
    mode = 'lines',
    name = 'ETP (m3/d)',
    marker=dict(
        color= 'green')
)
data.append(traceObs)

col = "sim"
for i in range(1, len(sims)):    
    filename = "resW_" + sims[i] + ".tss"
    sim_m3 = pd.read_table(sim_path + folder + back + filename, 
                                skiprows=4, delim_whitespace=True,
                                names=['Jdays', col],
                                header=None)
    etpSim = sim_m3.sim.tolist()
    trace = go.Scatter(
        x = x,
        y = etpSim,
        mode = 'lines',
        name = names[i],
        marker = dict(
            color= palette[i])
    )
    data.append(trace)


In [7]:
figname = "ETP.png"

%matplotlib inline
import plotlycred

online = False
if online:
    # fig = py.iplot(data) 
    fig = go.Figure(data=data) # , layout=layout)
    py.image.save_as(fig, filename=figname)
    # py.plot(fig, filename=figname) # Opens in web
else:
    fig = go.Figure(data=data) # , layout=layout)
    iplot(fig)


Follow this link to interact with the (old) data:

https://plot.ly/~DayTimeChunks/37/


In [8]:
from IPython.display import Image
if online:
    i = Image(figname)
    display(i)

In [9]:
name_tss = "cum_rain_obs_m3" 
filename = "resW_" + name_tss + ".tss"
col = "Rain"

# Rain (m3)
rain_m3 = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', col],
                            header=None)
rain_m3.head()

,Jdays,Rain
0,166,0.00
1,167,563.75
2,168,563.75
3,169,563.75
4,170,563.75


In [10]:
# Cumulative observed Q (m3/d)
name_tss = "cum_q_obs_m3"
filename = "resW_" + name_tss + ".tss"
col = "sim"
cumQ_obs = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', col],
                            header=None)
cumQ_obs.head()

,Jdays,sim
0,166,0.0
1,167,0.0
2,168,0.0
3,169,0.0
4,170,0.0


In [11]:
# Rain - Obs Q -> Theoretical ETP
etpTh = rain_m3.Rain - cumQ_obs.sim

In [12]:
# Cumulative sim Q (m3/d)
name_tss = "cum_q_sim_m3"
filename = "resW_" + name_tss + ".tss"
col = "sim"
cumQ_sim = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', col],
                            header=None)
cumQ_sim.head()

,Jdays,sim
0,166,0.0
1,167,0.0
2,168,0.0
3,169,0.0
4,170,0.0


In [13]:
name_tss = "totEtp_m3" 
filename = "resW_" + name_tss + ".tss"
col = "sim"

# Cum ETP (m3)
etpSum_m3 = pd.read_table(sim_path + folder + back + filename, 
                            skiprows=4, delim_whitespace=True,
                            names=['Jdays', col],
                            header=None)
etpSum_m3.head()

,Jdays,sim
0,166,0.0
1,167,0.0
2,168,0.0
3,169,0.0
4,170,0.0


In [14]:
if short_sim:
    Robs = rain_m3.Rain.tolist()
    Qobs = cumQ_obs.sim.tolist()
    Qsim = cumQ_sim.sim.tolist()
    ETPsim = etpSum_m3.sim.tolist()
    ETPTh = etpTh.tolist()
else:
    Robs = rain_m3.Rain.tolist()[s_obs:e_obs]
    Qobs = cumQ_obs.sim.tolist()[s_obs:e_obs]
    Qsim = cumQ_sim.sim.tolist()[s_obs:e_obs]
    ETPsim = etpSum_m3.sim.tolist()[s_obs:e_obs]
    ETPTh = etpTh.tolist()[s_obs:e_obs]

In [15]:
R = go.Scatter(
    x = x,
    y = Robs,
    mode = 'lines',
    name = 'Sum-Rain (m3/d)',
    marker=dict(
        color= 'green')
)

qObs = go.Scatter(
    x = x,
    y = Qobs,
    mode = 'lines',
    name = 'Sum-Q obs. (m3/d)',
    marker=dict(
        color= 'blue')
)

etpTh = go.Scatter(
    x = x,
    y = ETPTh,
    mode = 'lines',
    name = 'Theoretical ETP (m3/d)',
    marker=dict(
        color= 'red')
)

qSim = go.Scatter(
    x = x,
    y = Qsim,
    mode = 'lines',
    name = 'Sum-Q sim. (m3/d)',
    marker=dict(
        color= 'purple')
)

etpSim = go.Scatter(
    x = x,
    y = ETPsim,
    mode = 'lines',
    name = 'Sum-ETP sim. (m3/d)',
    marker=dict(
        color= 'orange')
)

data = [R, qObs, etpTh, qSim, etpSim]

In [16]:
figname = "SumDischarge.png"

%matplotlib inline
import plotlycred

online = False
if online:
    # fig = py.iplot(data) 
    fig = go.Figure(data=data) # , layout=layout)
    py.image.save_as(fig, filename=figname)
else:
    iplot(data)


In [17]:
cumrain_mm = 60
switch_day = cumrain_mm * 470
switch_day

28200

https://plot.ly/~DayTimeChunks/41/

In [18]:
from IPython.display import Image
if online:
    i = Image(figname)
    display(i)